## ATL03 Points with datashader

In [ ]:
import xarray as xr
import hvplot.xarray
import s3fs
import pandas as pd
import numpy as np

In [ ]:
# copy an ATL03 file to disk (I've put one from the ATL03/ATL06 tutorial in the hackweek s3 bucket)
!aws s3 cp s3://scottyhq/pangeo-demo/ATL03_20190928165055_00270510_003_01.h5 /srv/scratch/ATL03_20190928165055_00270510_003_01.h5

In [ ]:
!ls -lh /srv/scratch/ATL03_20190928165055_00270510_003_01.h5

In [ ]:
filePath='/srv/scratch/ATL03_20190928165055_00270510_003_01.h5'
group = 'gt1l/heights'
gt1l = xr.open_dataset(filePath, group=group, engine='h5netcdf')
gt1l

In [ ]:
npoints = int(gt1l.delta_time.size / 1e6)
print(f'This group contains ~{npoints} million points!')
print(f'group dataset size is ~{gt1l.nbytes/1e9:.2f} Gb')

In [ ]:
# Extract categorized detections
cols = ['h_ph', 'signal_conf_ph']

landice = gt1l[cols].sel(ds_surf_type=3)
land = gt1l[cols].sel(ds_surf_type=1)
land

In [ ]:
landice

In [ ]:
# Retain only 'noise' and 'signal' in this category of points
ds_landice = landice.where(landice.signal_conf_ph >=0, drop=True)
ds_land = land.where(land.signal_conf_ph >=0, drop=True)
ds_land

In [ ]:
# Above lines promote any int dtypes to float64 for some reason,,, can reassign as int8
ds_landice['signal_conf_ph'] = ds_landice.signal_conf_ph.astype('int8')
ds_land['signal_conf_ph'] = ds_land.signal_conf_ph.astype('int8')
ds_land

In [ ]:
nland = int(ds_land.delta_time.size / 1e6)
nlandice = int(ds_landice.delta_time.size / 1e6)
print(f'~{nland} million valid land points!')
print(f'~{nlandice} million valid landice points!')

In [ ]:
all_points = gt1l.h_ph.hvplot.scatter(x='delta_time', datashade=True, cmap='greys')
land_points = ds_land.h_ph.hvplot.scatter(x='delta_time', datashade=True, cmap='greens')
landice_points = ds_landice.h_ph.hvplot.scatter(x='delta_time', datashade=True, cmap='blues')
all_points * land_points * landice_points

In [ ]:
# extract variables of interest into a pandas dataframe
df = ds_landice[['h_ph','signal_conf_ph']].to_dataframe()
df.head()

In [ ]:
df.info()

In [ ]:
# Colorby signal strength flag 
import hvplot.pandas

colormap={0:'grey', 1: 'black', 2: 'yellow', 3:'orange', 4:'red' }
df.hvplot.scatter(x='delta_time', y='h_ph', by='signal_conf_ph', datashade=True, dynspread=True, cmap=colormap,
                  width=1000, height=500)